# 1 - Sequence to Sequence Learning with Neural Nertworks
- pytorch 와 torchtext를 활용해서 한 시퀀스에서 다른 시퀀스로 이동하는 머신러닝 모델을 구축할 것입니다.<br>
- 한글에서 영어로 번역할때 수행되지만 모델은 한 시퀀스에서 다른 시퀀스로 이동하는 것과 관련된 모든 문제에 적용될 수 있습니다. 즉, 동일한 언어의 시퀀스에서 더 짧은 시퀀스로 이동합니다.<br> 
- 첫 번째 노트북에서는 Neural Networks 논문의 모델을 구현하여 일반 개념을 이해하기 쉽게 할 것입니다.

## Introduction
- 가장 일반적인 시퀀스 대 시퀀스(seq2seq) 모델은 인코더-디코더 모델로 일반적으로 순환 신경망(RNN)을 사용하여 소스(입력) 문장을 단일 벡터로 인코딩합니다.
- 이 노트북에서는 이 단일 벡터를 context벡터라고 부를 것입니다. 컨텍스트 벡터는 전체 입력 문장의 추상적 표현으로 생각할 수 있습니다. 그런 다음 이 벡터는 한 번에 한 단어씩 생성하여 대상(출력)문장을 출력하는 방법을 학습하는 두 번째 RNN에 의해 디코딩됩니다.

<p align="center"><img src="https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png"></p>

- 위 이미지는 translation예시를 보여줍니다. 입력/소스 문장 "guten morgen"은 임베딩 레이어(노란색)를 통과한 다음 인코더(녹색)에 입력됩니다. 또한 시퀀스 시작(`<sos>`)및 시퀀스 끝(`<eos>`)토큰을 문장의 시작과 끝에 각각 추가합니다.
- 각 시간 단계에서 인코더RNN에 대한 입력은 현재 단어 e(xt)의 임베딩 e와 이전 시간 단계 ht-1 및 인코더 RNN의 숨겨진 상태입니다. 새로운 은닉 상태 ht를 출력합니다. 지금까지는 은닉 상태를 문장의 백터표현으로 생각할 수 있습니다. RNN은 다음 두 가지의 함수로 나타낼 수 있습니다. $e(x_{t})$ 및 $h_{t-1}$:
$$ h_{t} = EncoderRNN(e(x_{t}),h_{t-1})$$
여기서는 일반적으로 RNN이라는 용어를 사용합니다. LSTM또는 GRU와 같은 모든 반복 아키텍처가 될 수 있습니다.
우리는 $ X=(x_{1},x_{2},...,x_{T})$, 여기서 $x_{1}=<sos>,x_{2}=guten$ 등 초기 은닉 상태, $h_{0}$는 일반적으로 0 또는 학습된 매개변수로 초기화됩니다.
마지막 단어, $x_{T}$ 는 임베딩 레이어를 통해 RNN으로 전달되었으며 최종 은닉 상태를 사용합니다. $h_{T}$,컨텍스트 벡터, 즉 $h_{T}=z$ 이것은 전체 소스 문장의 벡터 표현입니다.
이제 컨텍스트 벡터 z가 있으므로 출력/목표 문장 "좋은 아침"을 얻기 위해 디코딩을 시작할 수 있습니다. 다시 말하지만, 시퀀스 토큰의 시작과 끝을 대상 문장에 추가합니다. 각 시간 단계에서 디코더 RNN(파란색)에 대한 입력은 이전 시간 단계의 숨겨진 상태뿐만 아니라 현재 단어 현재 단어 ($y_{t}$)의 임베딩, d 및 이전 시간 단계 st1의 숨겨진 상태이며, 여기서 초기 디코더 숨김 상태 s0은 컨텍스트 벡터, 즉 초기 디코더 숨김 상태 s0=z 이다. 따라서 인코더와 유사하게 우리는 디코더를 다음과 같이 나타낼 수 있다.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, Bucketiterator

import spacy
import math
import time

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))